In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pickle
import numpy as np
import random
from torch.utils.data.dataset import random_split

import sys
sys.path.insert(0, '../../')


In [2]:
random_seed = 123
torch.backends.cudnn.deterministic = True
torch.manual_seed(random_seed )
torch.cuda.manual_seed(random_seed )
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:

transform = transforms.Compose([transforms.ToTensor()])


train_dataset = dsets.MNIST(root='./data', train=True, transform=transform,download=True)


test_dataset = dsets.MNIST(root='./data', train=False, transform= transform)


In [4]:
class Lenet28(nn.Module):
    def __init__(self):
        super(Lenet28, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)  
        self.maxpool = nn.MaxPool2d(kernel_size=2)   
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)     
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.8) 
        
        self.fc1 = nn.Linear(64*7*7, 128) 
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 10)
        

    
    def forward(self, x):
 
        out = self.relu(self.cnn1(x))
        out = self.maxpool(out)
        out = self.relu(self.cnn2(out))
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.tanh(self.fc1(out))
        out = self.dropout(out)
        out = self.tanh(self.fc2(out))
        out = self.dropout(out)
        out = self.fc3(out)
  
        return out

In [5]:
# class Lenet64(nn.Module):
#     def __init__(self):
#         super(Lenet64, self).__init__()
        
#         self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)  
#         self.maxpool = nn.MaxPool2d(kernel_size=2)   
#         self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)     
        
#         self.relu = nn.ReLU()
#         self.tanh = nn.Tanh()
#         self.dropout = nn.Dropout(0.8) 
        
#         self.fc1 = nn.Linear(64*16*16, 128) 
#         self.fc2 = nn.Linear(128, 256)
#         self.fc3 = nn.Linear(256, 10)
        

    
#     def forward(self, x):
 
#         out = self.relu(self.cnn1(x))
#         out = self.maxpool(out)
#         out = self.relu(self.cnn2(out))
#         out = self.maxpool(out)
#         out = out.view(out.size(0), -1)
#         out = self.tanh(self.fc1(out))
#         out = self.dropout(out)
#         out = self.tanh(self.fc2(out))
#         out = self.dropout(out)
#         out = self.fc3(out)
  
#         return out

In [6]:
NUM_CLASSES = 10 
BATCH_SIZE = 100
NUM_EPOCHS = 10000

In [7]:
model = Lenet28()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

n_iters = NUM_EPOCHS * (len(train_dataset) / BATCH_SIZE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                       batch_size=BATCH_SIZE, 
                                       shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                  batch_size=BATCH_SIZE, 
                                  shuffle=False)

In [9]:
early_stopping = EarlyStopping(patience=5, verbose=False)
for epoch in range(NUM_EPOCHS):    
    train_loss = []
    valid_loss= [] 
    model.train()
    for images, labels in train_loader:
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() 
        train_loss.append(loss.item())

    model.eval()
    total = 0.0
    correct = 0.0
    for images, labels in test_loader:
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        total += labels.size(0)
        correct += (predicted.cpu() == labels.cpu()).sum()
        valid_loss.append(loss.item())
    correct = correct.float()
    accuracy = 100 * correct / total
    train_loss_avg = sum(train_loss) / len(train_loss)
    valid_loss_avg = sum(valid_loss) / len(valid_loss)



    print('Epoch: {}, Accuracy {}, train_loss : {}, test_loss : {}'.format(epoch, accuracy, train_loss_avg, valid_loss_avg))



# model.load_state_dict(torch.load('checkpoint.pt'))



NameError: name 'EarlyStopping' is not defined

In [ ]:
model.eval()

confusion_matrix = torch.zeros(NUM_CLASSES, NUM_CLASSES)
correct = 0
total = 0
for images, labels in test_loader:
    images = images.requires_grad_().to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    for t, p in zip(labels, predicted):
            confusion_matrix[t.long(), p.long()] += 1
    total += labels.size(0)
    correct += (predicted.cpu() == labels.cpu()).sum()
correct = correct.float()
accuracy = 100 * correct / total

per_class_accuracy = confusion_matrix.diag()/confusion_matrix.sum(1)

print('Acuracy: {}'.format(accuracy))